In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from transformers import BertTokenizer, BertModel
import torch
from sklearn.metrics.pairwise import cosine_similarity

import networkx as nx
from transformers import BertTokenizer, BertModel
from nltk.tokenize import sent_tokenize

In [41]:
# Read the data
df_neg = pd.read_json('/Users/williamfussell/Downloads/df_neg_sample.json')
df_pos = pd.read_json('/Users/williamfussell/Downloads/df_pos_sample.json')


In [42]:
# Inspect the data
df_neg.head()

,overall,asin,reviewText,summary,unixReviewTime,sentiment,title
0,1,B00004X145,Why would anyone make an oil drain pan with a ...,Difficult to pour oil out of this pan,1390003200,0,Flotool International 05070 7-1/2 Quart Round ...
1,1,B00004X145,Why would anyone make an oil drain pan with a ...,Difficult to pour oil out of this pan,1390003200,0,Flotool International 05070 7-1/2 Quart Round ...
2,1,B00004X145,this item is flimsy when filled with 5 quarts ...,dont buy this,1382227200,0,Flotool International 05070 7-1/2 Quart Round ...
3,1,B00004X145,this item is flimsy when filled with 5 quarts ...,dont buy this,1382227200,0,Flotool International 05070 7-1/2 Quart Round ...
4,2,B00004X145,I bought this because it was inexpensive and I...,Not my favorite,1355875200,0,Flotool International 05070 7-1/2 Quart Round ...


In [43]:
df_neu.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26284 entries, 1162 to 1709446
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   overall         26284 non-null  int64 
 1   asin            26284 non-null  object
 2   reviewText      26284 non-null  object
 3   summary         26284 non-null  object
 4   unixReviewTime  26284 non-null  int64 
 5   sentiment       26284 non-null  int64 
dtypes: int64(3), object(3)
memory usage: 1.4+ MB


In [44]:
df_pos.head()

,overall,asin,reviewText,summary,unixReviewTime,sentiment,title
0,4,B0000AXCCQ,Lots of storage. Not so easy to mount. All tra...,Does the job.,1401408000,2,Buyers Products 1701680 Black Poly Trailer Ton...
1,4,B0000AXCCQ,Lots of storage. Not so easy to mount. All tra...,Does the job.,1401408000,2,Buyers Products 1701680 Black Poly Trailer Ton...
2,5,B0000AXCCQ,I'm very happy with this purchase. Some of the...,Great box for the price!,1393459200,2,Buyers Products 1701680 Black Poly Trailer Ton...
3,5,B0000AXCCQ,I'm very happy with this purchase. Some of the...,Great box for the price!,1393459200,2,Buyers Products 1701680 Black Poly Trailer Ton...
4,5,B0000AXCCQ,This is a very good box for the money and it i...,Nice Box.,1498953600,2,Buyers Products 1701680 Black Poly Trailer Ton...


In [45]:
# Filter the DataFrame for a specific 'asin' value (product)
product_asin = df_neg['asin'][2]
product_reviews = df_neg[df_neg['asin'] == product_asin]['reviewText'].tolist()

# Load BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")

# Function to calculate BERT embeddings for a sentence
def get_bert_embedding(sentence):
    input_ids = tokenizer.encode(sentence, add_special_tokens=True)
    inputs = torch.tensor([input_ids])
    with torch.no_grad():
        outputs = model(inputs)
        return outputs.last_hidden_state.mean(dim=1).numpy()

# Calculate BERT embeddings for all sentences
sentence_embeddings = [get_bert_embedding(sentence) for sentence in product_reviews]

# Calculate cosine similarity between sentences
cosine_similarities = cosine_similarity(np.vstack(sentence_embeddings))

# Get the average cosine similarity score for each review
average_cosine_similarities = cosine_similarities.mean(axis=1)

# Select the indices of the top three reviews based on their average cosine similarity score
top_review_indices = np.argsort(average_cosine_similarities)[-num_sentences_in_summary:]

# Generate the extractive summary by selecting sentences from the top reviews
extractive_summary = []
for i in top_review_indices:
    sentences = product_reviews[i].split('. ')  # Split the review into sentences
    selected_sentences = sentences[:min(3, len(sentences))]  # Select up to 3 sentences
    extractive_summary.extend(selected_sentences)

# Create a string with extracted sentences enclosed in double quotes and separated by newline characters
formatted_summary = '\n'.join(['"' + sentence + '"\n' for sentence in extractive_summary])

# Print the formatted extractive summary
print(formatted_summary)
    


"I bought this because it was inexpensive and I thought 6 quarts was large enough"

"Unfortunately I did not think far enough ahead"

"My truck has 5qt oil capacity, and this container will hold a maximum of 6qts on a flat surface"

"I bought this because it was inexpensive and I thought 6 quarts was large enough"

"Unfortunately I did not think far enough ahead"

"My truck has 5qt oil capacity, and this container will hold a maximum of 6qts on a flat surface"



In [49]:
from rouge import Rouge

# Instantiate the ROUGE scorer
rouge = Rouge()

# Define your extractive summary and reference summary as strings
extractive_summary = (
"I bought this because it was inexpensive and I thought 6 quarts was large enough"

"Unfortunately I did not think far enough ahead"

"My truck has 5qt oil capacity, and this container will hold a maximum of 6qts on a flat surface"

"I bought this because it was inexpensive and I thought 6 quarts was large enough"

"Unfortunately I did not think far enough ahead"

"My truck has 5qt oil capacity, and this container will hold a maximum of 6qts on a flat surface"
)

reference_summary = (
    "The reviews express the buyer's initial satisfaction with the cost and perceived adequacy of a 6-quart container. However, they later realized its limitations, as their truck requires 5 quarts of oil and the container's maximum capacity is only slightly higher at 6 quarts, which might be insufficient for their needs. This realization comes from not having fully anticipated their requirements when making the purchase."
)

# Calculate ROUGE scores
scores = rouge.get_scores(extractive_summary, reference_summary)

# Access specific ROUGE metrics (e.g., ROUGE-2, ROUGE-L)
rouge_2_f1 = scores[0]["rouge-2"]["f"]
rouge_l_f1 = scores[0]["rouge-l"]["f"]

# Print the scores
print("ROUGE-2 F1 Score:", rouge_2_f1)
print("ROUGE-L F1 Score:", rouge_l_f1)

ROUGE-2 F1 Score: 0.0
ROUGE-L F1 Score: 0.19565216919896042


In [47]:
import networkx as nx
import torch
from transformers import BertTokenizer, BertModel
from nltk.tokenize import sent_tokenize
from sklearn.metrics.pairwise import cosine_similarity

# Filter the DataFrame for a specific 'asin' value (product)
product_asin = df_neg['asin'][2]
product_reviews = df_neg[df_neg['asin'] == product_asin]['reviewText'].tolist()

# Load BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")

# Function to calculate BERT embeddings for a sentence
def get_bert_embedding(sentence):
    input_ids = tokenizer.encode(sentence, add_special_tokens=True)
    inputs = torch.tensor([input_ids])
    with torch.no_grad():
        outputs = model(inputs)
        return outputs.last_hidden_state.mean(dim=1).numpy()

# Calculate BERT embeddings for all sentences
sentence_embeddings = [get_bert_embedding(sentence) for sentence in product_reviews]

# Flatten the 3D embeddings into 2D arrays
sentence_embeddings = [embedding.squeeze() for embedding in sentence_embeddings]

# Create a graph
G = nx.Graph()

# Calculate cosine similarity between BERT embeddings and add edges to the graph
num_sentences = len(product_reviews)
cosine_similarities = cosine_similarity(sentence_embeddings, sentence_embeddings)

# Tokenize sentences and preprocess
sentences = sent_tokenize(" ".join(product_reviews))

# Add nodes (sentences) to the graph
for i, sentence in enumerate(sentences):
    G.add_node(i, text=sentence)

# Add edges to the graph based on cosine similarities
for i in range(num_sentences):
    for j in range(i + 1, num_sentences):
        similarity = cosine_similarities[i][j]
        G.add_edge(i, j, weight=similarity)

# Calculate TextRank scores
scores = nx.pagerank(G)

# Select the top sentences based on TextRank scores
num_sentences_in_summary = 2  # Adjust as needed
summary_indices = sorted(range(len(sentences)), key=lambda i: scores[i], reverse=True)[:num_sentences_in_summary]
summary = [sentences[i] for i in summary_indices]

# Print the extractive summary
for sentence in summary:
    print(sentence)


Why would anyone make an oil drain pan with a spout that is flush with the top rim?
That's not a spout.


In [48]:
from rouge import Rouge

# Instantiate the ROUGE scorer
rouge = Rouge()

# Define your extractive summary and reference summary as strings
extractive_summary = (
    "Quality control is suspected."
    "Its just not as expected."
)

reference_summary = (
    "Customers express disappointment with this car vacuum, citing its poor performance and tendency to blow fuses. They find it lacks the suction power needed for effective cleaning, and some report issues with build quality, including a damaged cord and attachment problems. Despite its initial promise, this vacuum falls short of expectations, leaving buyers frustrated and dissatisfied."
)

# Calculate ROUGE scores
scores = rouge.get_scores(extractive_summary, reference_summary)

# Access specific ROUGE metrics (e.g., ROUGE-2, ROUGE-L)
rouge_2_f1 = scores[0]["rouge-2"]["f"]
rouge_l_f1 = scores[0]["rouge-l"]["f"]

# Print the scores
print("ROUGE-2 F1 Score:", rouge_2_f1)
print("ROUGE-L F1 Score:", rouge_l_f1)

ROUGE-2 F1 Score: 0.0
ROUGE-L F1 Score: 0.0
